<a href="https://colab.research.google.com/github/gokhangunal/AB_Testing_I/blob/main/AB_Testing_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [52]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [53]:
# Excel file path
excel_file = 'ab_testing.xlsx'

# Read the Control Group sheet
control_group = pd.read_excel(excel_file, sheet_name='Control Group')

# Read the Test Group sheet
test_group = pd.read_excel(excel_file, sheet_name='Test Group')

In [54]:
control_group.head(5)

,Impression,Click,Purchase,Earning
0,82529.46,6090.08,665.21,2311.28
1,98050.45,3382.86,315.08,1742.81
2,82696.02,4167.97,458.08,1797.83
3,109914.40,4910.88,487.09,1696.23
4,108457.76,5987.66,441.03,1543.72


In [55]:
test_group.head(5)

,Impression,Click,Purchase,Earning
0,120103.50,3216.55,702.16,1939.61
1,134775.94,3635.08,834.05,2929.41
2,107806.62,3057.14,422.93,2526.24
3,116445.28,4650.47,429.03,2281.43
4,145082.52,5201.39,749.86,2781.70


In [56]:
control_group.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00,101711.45,20302.16,45475.94,85726.69,99790.70,115212.82,147539.34
Click,40.00,5100.66,1329.99,2189.75,4124.30,5001.22,5923.80,7959.13
Purchase,40.00,550.89,134.11,267.03,470.10,531.21,637.96,801.80
Earning,40.00,1908.57,302.92,1253.99,1685.85,1975.16,2119.80,2497.30


In [57]:
test_group.describe().T

# Inference:
# The average number of ad impressions is higher in the test group.
# The average number of ad clicks from displayed ads is higher in the control group.
# The number of products purchased after clicking ads is higher in the test group.
# The amount of earnings obtained after purchasing products is higher in the test group.
# The control group represents the old method, which is maximum bidding.
# The test group represents the new method, which is average bidding.

,count,mean,std,min,25%,50%,75%,max
Impression,40.00,120512.41,18807.45,79033.83,112691.97,119291.30,132050.58,158605.92
Click,40.00,3967.55,923.10,1836.63,3376.82,3931.36,4660.50,6019.70
Purchase,40.00,582.11,161.15,311.63,444.63,551.36,699.86,889.91
Earning,40.00,2514.89,282.73,1939.61,2280.54,2544.67,2761.55,3171.49


In [58]:
# Add a new column named 'group' to the `control_group` DataFrame and set it to 'control'
control_group['group'] = 'control'

# Add a new column named 'group' to the `test_group` DataFrame and set it to 'test'
test_group['group'] = 'test'

# Vertically concatenate (stack) and use the 'group' column to indicate the source of the data
combined_df = pd.concat([control_group, test_group], axis=0)

# Reset the indices after concatenation
combined_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the combined DataFrame
print(combined_df.head())

   Impression   Click  Purchase  Earning    group
0    82529.46 6090.08    665.21  2311.28  control
1    98050.45 3382.86    315.08  1742.81  control
2    82696.02 4167.97    458.08  1797.83  control
3   109914.40 4910.88    487.09  1696.23  control
4   108457.76 5987.66    441.03  1543.72  control


In [59]:
######################################################
# A/B Testing (Independent Two-Sample T-Test)
######################################################

# 1. Establish Hypotheses
# 2. Assumption Check
#   - 1. Normality Assumption (Are the values normally distributed?)
#   - 2. Homogeneity of Variance (Are the variances of the values similar?)
# 3. Application of the Hypothesis
#   - 1. If assumptions are met, perform independent two-sample t-test (parametric test)
#   - 2. If assumptions are not met, perform Mann-Whitney U test (non-parametric test)
# 4. Interpret the results based on the p-value
# Note:
# - If normality is not met, proceed directly to step 2. If homogeneity of variance is not met, input argument in step 1 (e.g., say variances are not homogeneous).
# - Before examining normality, it can be beneficial to conduct and correct for outlier analysis.

In [ ]:
############################
# Application 1: Is there a statistically significant difference between the new bidding system implemented in the application and the old bidding system?
############################

In [60]:
############################
# 1. Establish the Hypothesis
############################

# H0: M1 = M2 (The means of the two groups are equal)
# H1: M1 != M2 (The means of the two groups are not equal)

############################
# 2. Assumption Check
############################

# Normality Assumption
# Homogeneity of Variance

############################
# Normality Assumption
############################

# H0: The assumption of normal distribution is satisfied.
# H1: The assumption of normal distribution is not satisfied.

In [61]:
combined_df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89
test,582.11


In [63]:
# We are entering into a mini hypothesis testing again. H0 states the distribution is normal. H1 states it is not normal. We test this using the Shapiro method.
# First, I'm testing this for the control group.
# If p-value < 0.05, then H0 is REJECTED.
# If p-value is not < 0.05, then H0 CANNOT BE REJECTED.
test_stat, pvalue = shapiro(combined_df.loc[combined_df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [41]:
# H0 cannot be rejected. Therefore, a normal distribution is achieved.

In [64]:
# Entering into another mini hypothesis test. H0 states the distribution is normal. H1 states the distribution is not normal. We are testing this using the Shapiro method.
# Now, I am testing this for the test group
# If p-value < 0.05, then H0 is REJECTED.
# If p-value is not < 0.05, then H0 CANNOT BE REJECTED.
test_stat, pvalue = shapiro(combined_df.loc[combined_df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [65]:
# H0 cannot be rejected. Therefore, a normal distribution is achieved.

In [44]:
# THE ASSUMPTION OF NORMAL DISTRIBUTION HAS BEEN MET.

# If the condition of normal distribution had not been met,
# I would not perform the variance test and would directly move to a Non-Parametric test.

In [45]:
############################
# Assumption of Homogeneity of Variance
############################
# Variance is the sum of the squares of deviations from the arithmetic mean. In other words, it is the standard deviation not taken to the square root.
# H0: The variances are homogeneous.
# H1: The variances are not homogeneous.

In [66]:
test_stat, pvalue = levene(combined_df.loc[combined_df["group"] == "control", "Purchase"],
                           combined_df.loc[combined_df["group"] == "test", "Purchase"])

# We use the Levene function to conduct the test for homogeneity of variances.

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [47]:
# If p-value < 0.05, then H0 is REJECTED.
# If p-value is not < 0.05, then H0 CANNOT BE REJECTED.

# H0 cannot be rejected. Therefore, the variances are homogeneous.

In [48]:
############################
# If assumptions are met, perform an independent two-sample t-test (parametric test)
############################

In [67]:
test_stat, pvalue = ttest_ind(combined_df.loc[combined_df["group"] == "control", "Purchase"],
                              combined_df.loc[combined_df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# The ttest_ind function says:
#   1. You can use me if both assumptions are met.
#   2. You can use me if only the normal distribution assumption is met.
#   3. You can use me with equal_var = False if the homogeneity of variance assumption is not met.
#      In this case, the Welch's test will be performed in the background.

# If p-value < 0.05, then H0 is REJECTED.
# If p-value is not < 0.05, then H0 CANNOT BE REJECTED.

Test Stat = -0.9416, p-value = 0.3493


In [ ]:
# H0 cannot be rejected. This means the statement that there is no statistically significant difference between the two group means is confirmed.
# Therefore, the implementation of the average bidding method has not created a statistically significant difference in purchases, and
# we cannot say that this new method has resulted in more purchases for our company.

In [ ]:
# H0 cannot be rejected. This means the statement that there is no statistically significant difference
# between the two group means has been confirmed.
# Thus, implementing the average bidding method has not created a statistically significant difference

In [ ]:
# Other statistical columns should also be examined to identify any significant differences.
# It cannot be claimed that the new method is ineffective based on a single column alone.